# CSM Simulation Groupwork Project

A bank operates with a single service counter. Customers arrive randomly and are served on a first-come, first-served basis. You are required to simulate this system using Python.

## Assumptions
- Inter-arrival times of customers are uniformly distributed between 1 and 8 minutes.
- Service times are uniformly distributed between 1 and 6 minutes.
- There is only one server (single-server queue).
- The queuing discipline is FIFO (First-In, First-Out).

## Task
Your group is expected to:
1. Generate inter-arrival and service times for 500 customers using the specified uniform
distributions.
2. Implement a Python simulation of the system using a discrete-event simulation approach.
3. Compute and report performance metrics.
4. Produce relevant visualizations.

## Setup
Create an environment and activate by running
 ```bash
 python -m venv venv
 venv\Scripts\activate
 pip install ipykernel
 ```

Then install the packages by running the command below:

In [71]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


## Code Implementation

Import the libraries

In [ ]:
import simpy
import random
import pandas as pd
import matplotlib.pyplot as plt

Create an array to store the arrival, service, service start, service end and wait times

And arrays for idle time calculation

In [73]:
simulation_data = [] # Store simulation data for each customer
last_service_end = [0] # List to store service end time for the last customer
total_idle_time = [0] # List to store total idle time for the server when new customer arrives

Created a function to simulate each individual customer

In [74]:
def customer(env, name, counter, inter_arrival_time):
    arrival_time = round(env.now, 2)

    with counter.request() as req:
        yield req
        service_start_time = round(env.now, 2)
        wait_time = round(env.now - arrival_time, 2)

        # Calculate idle time if any
        if service_start_time > last_service_end[0]:
            idle = round(service_start_time - last_service_end[0], 2)
            total_idle_time[0] += idle
        else:
            idle = 0

        service_time = round(random.uniform(1, 6), 2)
        yield env.timeout(service_time)

        service_end_time = round(env.now, 2)
        time_in_system = round(service_end_time - arrival_time, 2)

        # Update last_service_end
        last_service_end[0] = service_end_time

        simulation_data.append({
            'Customer': name,
            'Inter-Arrival Time': inter_arrival_time,
            'Arrival Time': arrival_time,
            'Wait Time': wait_time,
            'Service Time': service_time,
            'Service Start Time': service_start_time,
            'Service End Time': service_end_time,
            'Time in System': time_in_system,
            'Idle Time Before Service': idle
        })

Creating a function for Inter-Arrival Time (IAT) for a specific number of customers

In [75]:
max_customers = 500

In [76]:
def IAT(env, counter):
    current_customer = 0
    while current_customer < max_customers:
        # Generate inter-arrival time uniformly between 1 and 8 minutes
        inter_arrival_time = random.uniform(1, 8)
        # print(f'Inter-arrival time for customer {current_customer + 1}: {inter_arrival_time:.2f} minutes')
        yield env.timeout(inter_arrival_time)
        
        # Create a new customer
        current_customer += 1
        
        # Print and process the arrival of the customer
        # print(f'{customer} arrives at {env.now:.2f} minutes')
        env.process(customer(env, current_customer, counter, round(inter_arrival_time, 2)))

Setup environment and resource

In [77]:
env = simpy.Environment()
counter = simpy.Resource(env, capacity=1)  # Single server since we are doing MM1

Start arrival process

In [78]:
env.process(IAT(env, counter))  # Start the inter-arrival time process
env.run()

## Simulation Table

Printing the simulation table using a dataframe

In [80]:
df = pd.DataFrame(simulation_data)
df.head(10)

,Customer,Inter-Arrival Time,Arrival Time,Wait Time,Service Time,Service Start Time,Service End Time,Time in System,Idle Time Before Service
0,1,4.51,4.51,-0.00,2.36,4.51,6.87,2.36,4.51
1,2,2.27,6.78,0.09,4.53,6.87,11.40,4.62,0.00
2,3,3.56,10.34,1.06,4.71,11.40,16.11,5.77,0.00
3,4,7.04,17.37,0.00,4.92,17.37,22.29,4.92,1.26
4,5,4.27,21.64,0.65,5.67,22.29,27.96,6.32,0.00
5,6,3.40,25.04,2.92,1.97,27.96,29.93,4.89,0.00
6,7,2.86,27.90,2.03,1.24,29.93,31.17,3.27,0.00
7,8,5.71,33.61,0.00,1.76,33.61,35.37,1.76,2.44
8,9,6.74,40.35,0.00,2.44,40.35,42.79,2.44,4.98
9,10,6.24,46.59,-0.00,4.33,46.59,50.92,4.33,3.80
